# Google Colab 데이터 로드

In [ ]:
#Step 1. 구글 코랩에 한글 폰트 설정하기

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#Step 2.분석할 데이터가 저장된 파일을 불러와서 변수에 할당합니다.
from google.colab import files
myfile = files.upload()
import io
import pandas as pd
#pd.read_csv로 csv파일 불러오기
src_data = pd.read_csv(io.BytesIO(myfile['의사결정나무_과일종류_2가지.csv']),
                       encoding='cp949')
src_data

# 로컬 데이터 로드

In [1]:
#컴퓨터에서 작업하려면 아래 코드의 주석을 제거하고 실행하면 됩니다
import pandas as pd
src_data = pd.read_csv('../../머신러닝실습용자료/의사결정나무_과일종류_2가지.csv',encoding='cp949')
src_data

,종류,무게,길이
0,수박,2000,30.0
1,수박,2500,25.0
2,수박,1800,20.0
3,수박,1500,16.0
4,수박,1900,19.0
5,수박,600,9.0
6,참외,500,8.0
7,참외,400,7.5
8,참외,450,5.0
9,참외,400,4.5


# 공통 실습 코드

In [2]:
#Step 3.주어진 데이터를 훈련용 데이터와 검증용 데이터로 나눕니다
import numpy as np

# 무게, 길이로 종류를 예측
data = src_data[ ['무게','길이']].to_numpy() # np.column_stack( (src_data.무게, src_data.길이) )
target = src_data['종류'].to_numpy()
print(data)
print(target)

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
훈련용_data , 테스트용_data , 훈련용_target , 테스트용_target = train_test_split(
    data , target , test_size=0.2 , random_state=10)

[[2000.    30. ]
 [2500.    25. ]
 [1800.    20. ]
 [1500.    16. ]
 [1900.    19. ]
 [ 600.     9. ]
 [ 500.     8. ]
 [ 400.     7.5]
 [ 450.     5. ]
 [ 400.     4.5]
 [ 600.     9.5]
 [ 550.     8.5]]
['수박' '수박' '수박' '수박' '수박' '수박' '참외' '참외' '참외' '참외' '참외' '참외']


In [3]:
from sklearn.tree import DecisionTreeClassifier
# 의사결정트리 분류기 모델 생성
dt = DecisionTreeClassifier(random_state=10)

# 모델 학습
dt.fit(훈련용_data , 훈련용_target)

# 훈련용 데이터 기준 score 출력
print('훈련용 데이터 기준   정확도 :', dt.score(훈련용_data , 훈련용_target))

# 테스트용 데이터 기준 score 출력
print('테스트용 데이터 기준 정확도 :' , dt.score(테스트용_data , 테스트용_target))

훈련용 데이터 기준   정확도 : 1.0
테스트용 데이터 기준 정확도 : 1.0


In [ ]:
#3-Fold 교차 검증 수행
from sklearn.model_selection import cross_validate , cross_val_score
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=10)

# cross_validate() : 검사 결과를 자세하게 보여주는 것
scores_1 = cross_validate(dt , 훈련용_data , 훈련용_target, cv=3)

# cross_val_score() : 검사 결과 중에서 평균 점수만 보여주는 것
scores_2 = cross_val_score(dt, 훈련용_data , 훈련용_target, cv=3)

print('cross_validate 결과:', scores_1)
print('cross_validate 결과:', np.mean(scores_1['test_score']))
print('cross_val_score 결과:', np.mean(scores_2) )

In [ ]:
#4-Fold 교차검증 수행
from sklearn.model_selection import cross_validate , cross_val_score
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=10)
scores_1 = cross_validate(dt , 훈련용_data , 훈련용_target,cv=4)
scores_2 = cross_val_score(dt, 훈련용_data , 훈련용_target,cv=4)

print('cross_validate 결과:', scores_1)
print('cross_validate 결과:', np.mean(scores_1['test_score']))
print('cross_val_score 결과:', np.mean(scores_2) )

## GridSearch

### DecisionTree 하이퍼 파라미터
- max_depth :트리의 최대 깊이 (default: None)
- min_samples_split : 노드를 분할하기 위한 최소 샘플 데이터 수 (default: 2)
- max_features : 사용할 최대 피쳐 개수 ( default: None, 모든 피쳐 사용)
- min_samples_leaf : 분할될 때 자식 노드가 가져야할 최소 샘플 데이터 수 ( default: 1)
- **get_params()** 을 활용하여 하이퍼파라미터 확인 가능

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# 찾고 싶은 파라미터 입력
parm = {'max_depth':[1,2,3]}

# 의사결정트리에 그리드 서치 적용
gs = GridSearchCV(DecisionTreeClassifier(random_state=50) , param_grid=parm , n_jobs=-1)

# 그리드서치 학습
gs.fit(훈련용_data , 훈련용_target)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(estimator=DecisionTreeClassifier(random_state=50), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3]})

In [ ]:
# 최고의 성능을 보인 파라미터 출력
print(gs.best_params_)

# dt에 최고 성능 모델 복사
dt = gs.best_estimator_

# 훈련용_data로 dt의 score 출력
print(dt.score(훈련용_data , 훈련용_target))

In [7]:
#한꺼번에 여러 속성값을 찾을 경우

# 찾고싶은 파라미터 모두 정의
from sklearn.model_selection import GridSearchCV
parm = {'max_depth': range(2,10,1) ,
        'min_impurity_decrease': np.arange(0.0001, 0.001 , 0.0001),
        'min_samples_split' : range(2,10) }

# 의사결정트리에 그리드 서치 적용
gs = GridSearchCV(DecisionTreeClassifier(random_state=50) , parm , n_jobs=-1)

# 그리드서치 학습
gs.fit(훈련용_data , 훈련용_target)

# 최고의 성능을 보인 파라미터 출력
print(gs.best_params_)

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


{'max_depth': 2, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


In [8]:
# 그리드 서치 과정에서 교차검증 점수중 최고값을 확인하기
print(np.max(gs.cv_results_['mean_test_score']))

0.8


In [ ]:
# 최적의 모델로 테스트용 데이터로 최종 테스트하기
dt = gs.best_estimator_

# 훈련용, 테스트용 데이터로 수행한 score 출력
print(dt.score(훈련용_data , 훈련용_target))
print(dt.score(테스트용_data , 테스트용_target))